## 1. Upload data to APLO_SOURCE (aplo-source)

In [74]:
import glob, json, shutil, boto3, uuid, ast
from datetime import datetime, timedelta 
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient

az_des_con_str = 'DefaultEndpointsProtocol=https;AccountName=xetradestination;AccountKey=jHuTzDSICJbvjv/YKkQescXmPJg8Gntu/U1/P36fXqssoTAjPj7LCyo/lj1tTcjeSBaoY2IzFyKe+AStvRyTtQ==;EndpointSuffix=core.windows.net'
join_str = '.'
org_asep_oid = {
    "assigner": {
        "reference": "Apolo",
        "system": "OID_for_ASEP",
        "value": "urn:oid:"
    }
}

resource_list = ['Appointment','MedicationAdministration',
                 'Condition','Coverage','Medication',
                 'MedicationRequest','MedicationStatement',
                 'Observation','DiagnosticReport',
                 'AllergyIntolerance','Procedure']

# AWS Source
s3 = boto3.resource('s3')
bucket_target_source = s3.Bucket('aplo-source') 
client_del = boto3.client('s3')

# Azure Destination
blob_service_client = BlobServiceClient.from_connection_string(az_des_con_str)
container_client_destination = blob_service_client.get_container_client('aplo-destination')

# get look up file from the destination folder.
blob_list = container_client_destination.download_blob('look_up.json').readall()
blob_list = json.loads(blob_list)


In [75]:
def organization(entry,blob_service_client,blob_list):
    
    # Check if the organization already exists in lookup file then its a update senario.
    if len(entry['resource']['identifier']) > 0:
        
        if entry['resource']['identifier'][0]["assigner"]["value"] in blob_list['organization']:
            
            # it is an update senario
            # no changes just add update
            blob_client = blob_service_client.get_blob_client(
                container='aplo-destination', 
                blob=entry['resource']['identifier']["assigner"]["value"]+"/"+
                entry['resource']['resourceType']+
                "_"+entry['resource']['id']+"_"+str(uuid.uuid4())+".json")
            
            blob_client.upload_blob(json.dumps(entry),overwrite=True)

        else:
            
             # it is an invalid OID senario
            pass
    else:

        if len(blob_list['organization']) > 0:

            # lookup changes
            recent_prev_oid = blob_list['organization'][len(blob_list['organization']) - 1]
            recent_prev_oid_final_index = int(recent_prev_oid.split('.')[len(recent_prev_oid.split('.')) - 1])
            new_oid_final_index = recent_prev_oid_final_index + 1
            prefix_oid = join_str.join(recent_prev_oid.split('.')[0:len(recent_prev_oid.split('.')) - 1])
            current_oid = prefix_oid+"."+str(new_oid_final_index)
            blob_list['organization'].append(current_oid)


            # upload lookup
            blob_client = blob_service_client.get_blob_client(container='aplo-destination', blob='look_up.json')
            blob_client.upload_blob(json.dumps(blob_list),overwrite=True)


            # add the one more oid into the look up and add identifier into resource.
            org_asep_oid["assigner"]["value"] = org_asep_oid["assigner"]["value"] + current_oid
            entry['resource']['identifier'].append(org_asep_oid)
            blob_client = blob_service_client.get_blob_client(
                container='aplo-destination', 
                blob=current_oid+"/"+
                entry['resource']['resourceType']+
                "_"+entry['resource']['id']+"_"+str(uuid.uuid4())+".json")
            blob_client.upload_blob(json.dumps(entry),overwrite=True)

        else:

            # lookup changes
            blob_list['organization'].append('2.16.840.1.119358475923.1')
            # upload lookup
            blob_client = blob_service_client.get_blob_client(container='aplo-destination', blob='look_up.json')
            blob_client.upload_blob(json.dumps(blob_list),overwrite=True)


            # add the one more oid into the look up and add identifier into resource.
            org_asep_oid["assigner"]["value"] = org_asep_oid["assigner"]["value"] + "2.16.840.1.119358475923.1"
            entry['resource']['identifier'].append(org_asep_oid)


            # upload org data
            blob_client = blob_service_client.get_blob_client(
                container='aplo-destination', 
                blob="2.16.840.1.119358475923.1"+"/"+
                entry['resource']['resourceType']+
                "_"+entry['resource']['id']+"_"+str(uuid.uuid4())+".json")
            blob_client.upload_blob(json.dumps(entry),overwrite=True)


In [ ]:
def practitioner(blob_service_client, entry):
    blob_client = blob_service_client.get_blob_client(
                container='aplo-destination', 
                blob="2.16.840.1.119358475923.1"+"/"+
                entry['resource']['resourceType']+
                "_"+entry['resource']['id']+"_"+str(uuid.uuid4())+".json")
    blob_client.upload_blob(json.dumps(entry),overwrite=True)



In [76]:
def main(blob_service_client,blob_list,bucket_target_source):
    # read all data from source 
    
    for obj in bucket_target_source.objects.all():
        data_obj = bucket_target_source.Object(key=obj.key).get().get('Body').read().decode('utf8').replace("'",'"')
        data_obj = json.loads(data_obj)

        for entry in data_obj['entry']:

            if str(entry['resource']['resourceType']).title() in resource_list:
                pass

            elif str(entry['resource']['resourceType']).title() == 'Organization':
                organization(entry,blob_service_client,blob_list)   

            elif str(j['resource']['resourceType']).title() =='Practitioner':
                practitioner(blob_service_client, entry)

            else:
                pass

        client_del.delete_object(Bucket='aplo-source', Key=obj.key)

In [77]:
main(blob_service_client,blob_list,bucket_target_source)

In [78]:
# get look up file from the destination folder.
blob_list = container_client_destination.download_blob('look_up.json').readall()
blob_list = json.loads(blob_list)
blob_list

{'organization': ['2.16.840.1.119358475923.1',
  '2.16.840.1.119358475923.2',
  '2.16.840.1.119358475923.3',
  '2.16.840.1.119358475923.4',
  '2.16.840.1.119358475923.5',
  '2.16.840.1.119358475923.6',
  '2.16.840.1.119358475923.7',
  '2.16.840.1.119358475923.8',
  '2.16.840.1.119358475923.9'],
 'practitioner': [],
 'patient': []}